In [16]:
import pandas as pd


单品名	成本	亏损比	销量约束最小值	销量约束最大值	成本加成定价约束最小值	成本加成定价约束最大值	补货量最小值

In [17]:
df = pd.read_excel("中间数据集\\3_2_7月1日单品寻优的约束条件.xlsx")
name = df.columns
df.head()

,单品名称,成本,损耗率,销量约束min,销量约束max,定价约束min,定价约束max,最小补货量
0,白玉菇(袋),3.29,6.57,0.80,1.2000,3.28,7.497,2.675800
1,菠菜,9.66,18.51,1.02,3.9024,11.20,16.800,3.067861
2,菠菜(份),4.07,9.43,0.00,24.0000,0.00,7.188,2.760296
3,菜心,4.62,13.70,0.00,4.9344,0.00,7.200,2.896871
4,虫草花(份),2.60,9.43,0.80,4.8000,1.84,4.560,2.760296


Q = 成本加成定价($F_1$) * 补货量($N$) * (1-k) - 成本($F_2$) * 补货量($N$)
  = 补货量($N$) * [ (1-k) * 成本加成定价($F_1$) ]
 
$Q = F_1 * N_1 - 4.36 * N_2$

$-16\leq F_1 \leq 45.6$

$0\leq N_1 \leq 1.2$

$N_2 \geq 2.6758$

In [18]:
df.head()

,单品名称,成本,损耗率,销量约束min,销量约束max,定价约束min,定价约束max,最小补货量
0,白玉菇(袋),3.29,6.57,0.80,1.2000,3.28,7.497,2.675800
1,菠菜,9.66,18.51,1.02,3.9024,11.20,16.800,3.067861
2,菠菜(份),4.07,9.43,0.00,24.0000,0.00,7.188,2.760296
3,菜心,4.62,13.70,0.00,4.9344,0.00,7.200,2.896871
4,虫草花(份),2.60,9.43,0.80,4.8000,1.84,4.560,2.760296


In [19]:
def find_best(i):
    score, score_F, score_N_buhuo, name = [], [], [], []

    # F(成本加成定价)的取值在df的第6和第7列之间
    for F in range(int(df.iloc[i, 5] * 10), int(df.iloc[i, 6] * 10 + 1)):
        F = F / 10.0

        # 如果销售约束最小值大于等于0
        if df.iloc[i, 3] >= 0:


            # N(销售量)的取值在df的第4列和第5列之间
            for N in range(int(df.iloc[i, 3] * 10), int(df.iloc[i, 4] * 10 + 1)):  
                N = N / 10.0  

                # 如果销量比最小展出量小，那补货量就是df的第八列（最小补货量）
                if N <= 2.5:
                    score.append(F * N - df.iloc[i, 1] * df.iloc[i, 7])
                    score_N_buhuo.append(df.iloc[i, 7])
                    score_F.append(F)
                    name.append(df.iloc[i, 0])
                # 如果销量比最小展出量大，那补货量就是 N / (1 - 损耗率)
                else:
                    score.append(F * N - df.iloc[i, 1] * (N / (1 - df.iloc[i, 2] / 100)))
                    score_N_buhuo.append(N / (1 - df.iloc[i, 2] / 100))
                    score_F.append(F)
                    name.append(df.iloc[i, 0])

        else:


            for N in range(0, int(df.iloc[i, 4] * 10 + 1)):  
                N = N / 10.0  

                # 如果销量比最小展出量小，那补货量就是df的第八列（最小补货量）
                if N <= 2.5:
                    score.append(F * N - df.iloc[i, 1] * df.iloc[i, 7])
                    score_N_buhuo.append(df.iloc[i, 7])
                    score_F.append(F)
                    name.append(df.iloc[i, 0])
                # 如果销量比最小展出量大，那补货量就是 N / (1 - 损耗率)
                else:
                    score.append(F * N - df.iloc[i, 1] * (N / (1 - df.iloc[i, 2] / 100)))
                    score_N_buhuo.append(N / (1 - df.iloc[i, 2] / 100))
                    score_F.append(F)
                    name.append(df.iloc[i, 0])


    max_score_index = score.index(max(score))
    max_F = score_F[max_score_index]
    max_N = score_N_buhuo[max_score_index]
    max_name = name[max_score_index]

    #print(name[i])
    #print("最大的score值:", max(score))
    #print("对应的F1值:", max_F)
    #print("对应的N1值:", max_N)

    return max(score), max_F, max_N, max_name

<br><br>

In [20]:
score, F_score, N_score, label = [], [], [], []
for i in range(0, len(df)):
    S, F, N, names = find_best(i)
    score.append(S)
    F_score.append(F)
    N_score.append(N)
    label.append(names)

In [21]:
df = pd.DataFrame({'菜名': label, '最佳收益': score, '成本加成定价': F_score, '补货量': N_score})
df.head()

,菜名,最佳收益,成本加成定价,补货量
0,白玉菇(袋),0.076618,7.4,2.675800
1,菠菜,19.288561,16.8,4.785863
2,菠菜(份),62.549718,7.1,26.498841
3,菜心,9.048250,7.2,5.677868
4,虫草花(份),7.820603,4.5,5.299768


In [22]:
# 按照'Score'列的值从大到小排序
sorted_df = df.sort_values(by='最佳收益', ascending=False)
sorted_df.head(33)

,菜名,最佳收益,成本加成定价,补货量
41,云南生菜(份),212.045234,6.9,76.846638
30,西兰花,170.395927,16.8,22.261406
34,小米椒(份),101.784286,4.9,43.723087
45,枝江青梗散花,100.988389,16.8,14.795186
31,西峡花菇(1),88.227444,28.8,8.744395
27,娃娃菜,81.808794,8.1,23.379820
48,紫茄子(2),77.708745,7.2,23.315235
44,长线茄,72.252632,14.4,11.278195
14,净藕(1),69.262117,18.6,9.633707
43,云南油麦菜(份),67.526280,5.3,34.448493


In [23]:
sortef_df_33 = sorted_df.iloc[0:33, :]
print("列的总和:", sortef_df_33['最佳收益'].sum())

列的总和: 2002.8594042084244


In [24]:
sorted_df.to_excel("结果\\3_全体单品寻优情况.xlsx", index=None)
sortef_df_33.to_excel("结果\\3_前33位单品寻优情况.xlsx", index=None)